<a href="https://colab.research.google.com/github/bdlemos/Gemini_RAG/blob/main/Rag_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ChatBot com Generative AI utilizando o LangChain e o modelo gemini 1.0-pro**

O projeto tem como objetivo receber diversos arquivos pdf ou txt do usúario e, a partir desses arquivos, aprender sobre as informações contidas e conseguir ter um resultado mais confiáveis e embasados para as perguntas recebidas. A técnica utilizada para esse trabalho é a técnica demoninada RAG, que é um paradigma no campo do Processamento de Linguagem Natural (PLN) que combina técnicas de geração de texto com recuperação de informações.


**Instruçoes de uso:** Criar uma pasta chamada Dataset e por nela todos os documentos que quiser passar pra LLM

# Definição do modelo, preparação da informação e todo necessario para funcionar

### Iniciando

Baixando bibliotecas necessarias


In [ ]:
!apt-get install poppler-utils \
tesseract-ocr -y

!pip install --upgrade --quiet  langchain-google-genai
!pip install -q -U google-generativeai
!pip install faiss-gpu
!pip install sentence_transformers
!pip install google-cloud-aiplatform==1.46.0 \
'bigframes<1.0.0' \
langchain==0.1.14 \
unstructured==0.12.6 \
pillow-heif==0.15.0 \
unstructured-inference==0.7.25 \
pypdf==4.1.0 \
pdf2image==1.17.0 \
unstructured_pytesseract==0.3.12 \
pikepdf==8.14.0 \
--upgrade \
--user --quiet

### Variaveis

Variveis que indicam os diretorios e modelo utilizado ao longo do código

In [2]:
INDEX_PATH = "./Dataset/"

TEXT_MODEL = "gemini-1.0-pro"

### Importando modulos


In [3]:
# Utils
import os
import time
from typing import List

# Langchain
import langchain
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import TextLoader, UnstructuredPDFLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

print(f"LangChain version: {langchain.__version__}")


LangChain version: 0.1.14


### Data Preparation

Os arquivos pdf que queremos utilizar para embasar as respostas devem ser colocado no diretorio "./Dataset"

### Definindo funçoes utilitárias

**Carregamento de Documentos**: Leia o conteúdo dos documentos do diretório fornecido.

**Divisão de Documentos**:
Cria um objeto CharacterTextSplitter com os parâmetros:

**chunk_size=8192**: Visando pedaços de texto de cerca de 8192 caracteres.
**chunk_overlap=128**: Uma pequena sobreposição entre os pedaços, provavelmente para preservar o contexto ao dividir os documentos.

Aplica o text_splitter aos documentos carregados, dividindo-os em pedaços de texto menores e mais gerenciáveis.

Reúne todos os pedaços de texto divididos em uma única lista e retorna para processamento posterior

In [4]:
def get_split_documents(index_path: str) -> List[str]:
    """
    This function is used to chunk documents and convert them into a list.

    Args:
    index_path: Path of the dataset folder containing the documents.

    Returns:
    List of chunked, or split documents.
    """

    split_docs = []

    for file_name in os.listdir(index_path):
        print(f"file_name : {file_name}")
        if file_name.endswith(".pdf"):
            loader = UnstructuredPDFLoader(index_path + file_name)
        elif file_name.endswith(".ipynb_checkpoints"):
            continue
        else:
            loader = TextLoader(index_path + file_name)

        text_splitter = CharacterTextSplitter(chunk_size=8192, chunk_overlap=128)
        split_docs.extend(text_splitter.split_documents(loader.load()))

    return split_docs

### Create Vector Database


**Carregar Documentos:** A função get_split_documents recupera e pré-processa documentos do caminho especificado em INDEX_PATH.

**Gerar Incorporações Vetoriais:** O código gera incorporações vetoriais.

**Criar Banco de Dados Vetoriais:** Um banco de dados vetoriais FAISS (db) é inicializado. Este banco de dados especializado é projetado para armazenar e pesquisar eficientemente as incorporações vetoriais.

In [7]:
split_docs = get_split_documents(INDEX_PATH)
embedding = HuggingFaceEmbeddings()
db = FAISS.from_documents(split_docs,embedding)

file_name : 2111.01137v1.pdf


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


### Carregando o modelo

In [8]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('secretName')

In [9]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }


In [10]:
from langchain_google_genai import GoogleGenerativeAI, HarmBlockThreshold, HarmCategory

llm = GoogleGenerativeAI(model='gemini-1.0-pro',
                         google_api_key=GOOGLE_API_KEY,
                         generation_config=generation_config,)

### Criando um recuperador utilizando o armazenamento de vetores FAISS para busca de similaridade.

search_kwargs={"k": 3} - Este parâmetro, específico para busca de similaridade, indica que o recuperador deve retornar os 3 documentos mais relevantes com base nos escores de similaridade calculados.

In [11]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

Define um modelo de prompt para um modelo de linguagem que consiste em um modelo de string. Ele aceita um conjunto de parâmetros do usuário que podem ser usados para gerar um prompt para um modelo de linguagem

In [12]:
template = """
You are a helpful AI assistant. You're tasked to answer the question given below, but only based on the context provided.
context:
<context>
{context}
</context>

question:
<question>
{input}
</question>

If you cannot find an answer ask the user to rephrase the question.
answer:

"""
prompt = PromptTemplate.from_template(template)


### Criando a cadeia de recuperação e a invoque passando a pergunta como entrada.

In [13]:
# Create the retrieval chain
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

# Uso

# Exemplo de uso

In [21]:
# Invoke the retrieval chain
response = retrieval_chain.invoke({"input": "What are the methods used in this paper of stock prediction"})

In [22]:
print(response["answer"])

- Holt-Winters Exponential Smoothing
- ARIMA
- Random Forest
- MARS
- RNN
- LSTM


## Interface de uso

In [19]:
import ipywidgets as widgets
from IPython.display import clear_output

button = widgets.Button(description="Ask Me!")
output = widgets.Output()
button_stp = widgets.Button(description="More details")
output = widgets.Output()
text = widgets.Text(
    description="Question:", layout=widgets.Layout(width="80%", height="50px")
)
display(text, button, button_stp, output)


@output.capture()
def on_button_clicked(b):
    clear_output()
    question = text.value

    result = retrieval_chain.invoke({"input": question})
    source_documents = list({doc.metadata["source"] for doc in result["context"]})

    print("\nAnswer-", result["answer"])
    print("\nSource-", "\n".join(source_documents))
    print("\n")


@output.capture()
def on_stp_clicked(b):
    clear_output()
    question = text.value
    query = question + "Give detailed information as much as possible. "
    result = retrieval_chain.invoke({"input": query})
    source_documents = list({doc.metadata["source"] for doc in result["context"]})

    print("\nAnswer-", result["answer"])
    print("\nSource-", "\n".join(source_documents))
    print("\n")


button.on_click(on_button_clicked)
button_stp.on_click(on_stp_clicked)

Text(value='', description='Question:', layout=Layout(height='50px', width='80%'))

Button(description='Ask Me!', style=ButtonStyle())

Button(description='More details', style=ButtonStyle())

Output()